In [28]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
# import hvplot.dask
import numpy as np
import dask.array as da
import pickle
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [29]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [30]:
# df = pd.read_csv("data/training_set_VU_DM.csv")
df = pd.read_csv("data/train_train.csv")

In [31]:
original_len = len(df)
original_len

4711724

In [32]:
# 1. Probability if click then book for hotel, if search then click and if search then book
def click_book_df(df):
    grouped_hotels = df.groupby("prop_id").sum(numeric_only=True)
    nr_hotel_found = df["prop_id"].value_counts()
    temp = pd.DataFrame()
    temp["prob_clickbook"] = grouped_hotels["booking_bool"]/ grouped_hotels["click_bool"]
    temp["prob_searchbook"] = grouped_hotels["booking_bool"]/nr_hotel_found
    temp["prob_searchclick"] = grouped_hotels["click_bool"]/nr_hotel_found
    temp.fillna(0,inplace=True)
    temp =temp.reset_index()
    return temp

temp = click_book_df(df)
df = df.merge(temp, on="prop_id")
del temp

In [33]:
# 2. Percentile rank of hotel in its country
def hotel_country_rank_df(df):
    grouped_hotels = df.groupby(["prop_id","prop_country_id"]).mean(numeric_only=True).reset_index()
    temp = grouped_hotels[["prop_starrating", "prop_review_score","prop_id","prop_country_id"]].copy()
    temp["total_score"] = temp.loc[:,"prop_starrating"] + temp.loc[:,"prop_review_score"]
    temp["country_pct_rank"] = temp.groupby("prop_country_id")["total_score"].rank(method="dense",
                                                                               ascending=True,pct=True)
    temp = temp[["prop_id","country_pct_rank"]]
    return temp

temp = hotel_country_rank_df(df)
df = df.merge(temp, on="prop_id")
del temp

In [34]:
# 3. Competitors: mean competitor avail/ rate + if there is a better competitor columns.


def competitor_cols(temp:pd.DataFrame, col_names:list,rate_or_avail:str):
    """
    Compute 2 columns for rate and availability. Boolean column if there exists
    a competitor with a better rate and mean competitor values.
    """
    temp_rate = temp[col_names].copy()
    temp_rate_binary = temp_rate.applymap(lambda x: 1 if not pd.isna(x) and x >= 0 else 0)
    temp_rate_binary.loc[:, f"comp_mean_{rate_or_avail}"] = temp_rate.mean(axis=1)
    temp_rate_binary[f"comp_better_{rate_or_avail}"] = temp_rate_binary.any(axis=1).astype(int) #
    temp_rate_binary.drop(columns=col_names,inplace=True)
    return temp_rate_binary

def competitor_df(df):
    inv_cols = [f"comp{i}_inv" for i in np.arange(1,9)]
    rate_cols = [f"comp{i}_rate" for i in np.arange(1,9)]
    rate_perc_cols = [f"comp{i}_rate_percent_diff" for i in np.arange(1,9)]
    
    temp = df.groupby("prop_id").mean(numeric_only=True)
    rate_res = competitor_cols(temp, rate_cols, "rate")
    inv_res = competitor_cols(temp, inv_cols, "inv")
    temp = pd.concat([rate_res, inv_res],axis=1) # only new columns
    return temp

temp = competitor_df(df)
df = df.merge(temp, on="prop_id")# df = 

del temp

# drop original competitor columns
inv_cols = [f"comp{i}_inv" for i in np.arange(1,9)]
rate_cols = [f"comp{i}_rate" for i in np.arange(1,9)]
rate_perc_cols = [f"comp{i}_rate_percent_diff" for i in np.arange(1,9)]

# df.drop(columns=rate_cols+inv_cols+rate_perc_cols, inplace=True)

In [35]:
# 5. Price change in percentile, no NaN so don't need to average
def price_change_col(df):
    old_price = np.exp(df["prop_log_historical_price"])
    return (df["price_usd"] - old_price) / old_price
    
df["price_change"] = price_change_col(df)


In [36]:
# 6. Desirability_score as function of location and affinity score

# Compute the desirability score as a weighted average of the four measures
def desirability_col(df):
    desirability_score = np.exp(df["srch_query_affinity_score"]) *  (df["prop_location_score1"] + df["prop_location_score2"])
    desirability_score.fillna(desirability_score.mean(), inplace=True)
    return desirability_score

df["desirability_score"] = desirability_col(df)

In [37]:
# 7. Date time to percentile of the day

def percent_day_col(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    return (df['date_time'].dt.hour * 3600 + df['date_time'].dt.minute * 60 + df['date_time'].dt.second) / 86400

def month_col(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    return df["date_time"].dt.month

def weekend_bool_col(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    return df["date_time"].dt.day_of_week

df['percentile_of_day'] = percent_day_col(df)
df['month'] = month_col(df)
df['day_of_week'] = weekend_bool_col(df)
# df.drop(columns=["date_time"], inplace=True)

In [38]:
df

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,prob_clickbook,prob_searchbook,prob_searchclick,country_pct_rank,comp_mean_rate,comp_better_rate,comp_mean_inv,comp_better_inv,price_change,desirability_score,percentile_of_day,month,day_of_week
0,0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.95,27,104.77,0,23246,1,0,4,0,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.017212,0.027539,0.65,-0.012095,1,-0.009482,1,-0.257871,0.001458,0.355729,4,3
1,6938,498,2012-12-23 11:32:22,32,220,NaN,NaN,219,893,3,3.5,1,2.83,0.0950,4.96,28,88.37,0,25964,1,0,1,0,1,0,NaN,7500.57,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.017212,0.027539,0.65,-0.012095,1,-0.009482,1,-0.380268,0.001458,0.480810,12,6
2,12605,896,2013-05-09 22:14:57,5,2,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.98,24,126.00,0,23246,1,16,1,0,1,1,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,0.625,0.017212,0.027539,0.65,-0.012095,1,-0.009482,1,-0.133868,0.001458,0.927049,5,3
3,21958,1495,2013-04-24 02:24:02,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.98,28,139.00,0,23246,1,40,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.625,0.017212,0.027539,0.65,-0.012095,1,-0.009482,1,-0.044505,0.001458,0.100023,4,2
4,30512,2052,2013-01-11 08:10:27,5,219,NaN,NaN,219,893,3,3.5,1,2.83,0.0438,4.96,29,109.00,0,23246,1,0,2,0,1,1,NaN,77.22,0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,6.0,0,NaN,0,0.625,0.017212,0.027539,0.65,-0.012095,1,-0.009482,1,-0.235591,0.001458,0.340590,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711719,4957427,332713,2013-04-15 20:51:59,12,99,NaN,NaN,99,103277,3,5.0,0,0.69,0.0242,4.85,2,165.54,0,18778,2,4,2,0,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000,0.000000,0.000000,0.80,0.000000,1,0.000000,1,0.295910,0.001458,0.869433,4,0
4711720,4957430,332713,2013-04-15 20:51:59,12,99,NaN,NaN,99,135856,4,3.5,0,2.30,0.1415,5.38,1,155.11,0,18778,2,4,2,0,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.000,0.000000,0.000000,0.75,0.000000,1,0.000000,1,-0.285281,0.001458,0.869433,4,0
4711721,4957806,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,1668,1,3.5,1,0.00,0.0000,4.05,1,65.42,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,261.68,1,1.000,1.000000,1.000000,0.45,0.000000,1,0.000000,1,0.139772,0.001458,0.512488,2,0
4711722,4957807,332740,2013-02-18 12:17:59,12,99,NaN,NaN,99,2284,0,0.0,0,0.00,NaN,4.82,19,44.06,0,21461,4,0,1,0,1,0,NaN,NaN,0,NaN,NaN,NaN,Na

In [39]:
def usa_col(df):
    # Encode visitor/hotel from USA (approx. 50% of visitors)
    # idx visitor  hotel  encoding
    # 0     1     1        2
    # 1     0     0        0
    # 2     1     0        1
    # 3     0     1       -1
    visitor_us = (df["visitor_location_country_id"] == 219).astype(int)
    hotel_us = (df["prop_country_id"] == 219).astype(int)
    return np.where(visitor_us != hotel_us, visitor_us - hotel_us, hotel_us*2)

df["within_us"] =  usa_col(df)

In [40]:
# df.to_csv("data/FE_training_set.csv")
df.to_csv("data/train_train_FE.csv")

## Test set

- need to add expected click stuff here too, but using values from train set bc. dont have test set values

In [59]:
# df_test = pd.read_csv("data/test_set_VU_DM.csv")
df_test = pd.read_csv("data/train_test.csv")

In [60]:
len_test_original = len(df_test)
len_test_original

246623

In [61]:
df_train = df.copy(deep=True)

#### Copy click, search, booking columns to test dataset
Dataset with len(unique hotels that are in both).

With only the new columns to copy them to the test data for the matching hotels.

In [62]:
test_props = df_test["prop_id"].unique()
train_props = df_train["prop_id"].unique() 

# training dataset, but only the properties that are in the test set too
train_df_intersect = df_train[df_train["prop_id"].isin(np.intersect1d(test_props,train_props))]  

# select the columns we added before
clicks_train = train_df_intersect[["prop_id","prob_clickbook","prob_searchbook",
                                      "prob_searchclick","country_pct_rank"]]

# reduce the nr of rows to unique entries
clicks_train= clicks_train.groupby("prop_id").mean() 
clicks_train["in_train"] = 1 # bool column if was already in training set
clicks_train.reset_index(inplace=True)

In [63]:
df_test = df_test.merge(clicks_train, on="prop_id",how="left") 

In [64]:
df_test["in_train"].fillna(0, inplace=True) # if property not in training set, fill with 0

##### Rank of the hotel must be computed using both the test and training set

#### All other features must be computed for all rows in the test set
- compute hotel rank with whole test set, but only update hotels that are new

In [65]:
# add ranking 
save = df_test.copy()
len(save)

246623

In [66]:
df_test = save.copy()

In [67]:
# apply ranking, filter out new hotels and appy only there
temp = hotel_country_rank_df(df_test)
hotels_not_in_train = df_test[df_test["in_train"]==0]["prop_id"].unique()
temp = temp[temp["prop_id"].isin(hotels_not_in_train)]

# fill nan values of the missing ranks_rank_x","country_pct_rank_y"], inplace=True)

In [68]:
df_test = df_test.merge(temp,on="prop_id",how="left")

In [69]:
#  for the new hotels with the values from the testranking
df_test = df_test.merge(temp,on="prop_id",how="left")
df_test["country_pct_rank"] = df_test['country_pct_rank_x'].combine_first(df_test['country_pct_rank_y'])
df_test.drop(columns=["country_pct_rank_x","country_pct_rank_y"],inplace=True)

In [70]:
df_test = df_test.merge(competitor_df(df_test), on="prop_id")

In [71]:
df_test["price_change"] = price_change_col(df_test)
df_test["desirability_score"] = desirability_col(df_test)
df_test['percentile_of_day'] = percent_day_col(df_test)
df_test['month'] = month_col(df_test)
df_test['day_of_week'] = weekend_bool_col(df_test)
df_test["within_us"] =  usa_col(df_test)

In [72]:
df_test

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,prob_clickbook,prob_searchbook,prob_searchclick,in_train,country_pct_rank,comp_mean_rate,comp_better_rate,comp_mean_inv,comp_better_inv,price_change,desirability_score,percentile_of_day,month,day_of_week,within_us
0,371,39,2013-04-06 18:08:58,5,219,NaN,NaN,219,4426,3,4.0,1,1.61,NaN,4.66,9,135.00,0,24361,1,7,2,2,1,1,NaN,43.34,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,0.021505,0.021505,1.0,0.700000,0.0,1,0.0,1,0.277972,0.001267,0.756227,4,5,2
1,485026,32446,2013-03-05 13:19:18,14,100,NaN,NaN,219,4426,3,4.0,1,1.61,0.0415,4.66,18,98.35,0,17747,3,23,2,0,1,1,NaN,1520.03,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,1.0,0.021505,0.021505,1.0,0.700000,0.0,1,0.0,1,-0.068973,0.001267,0.555069,3,1,-1
2,1862780,125024,2013-06-29 14:50:06,5,219,NaN,NaN,219,4426,3,4.0,1,1.61,0.0415,0.00,4,120.00,0,17747,1,14,1,0,1,1,NaN,380.98,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,1.0,0.021505,0.021505,1.0,0.700000,0.0,1,0.0,1,119.000000,0.001267,0.618125,6,5,2
3,3834846,257454,2013-04-27 20:16:05,5,219,NaN,NaN,219,4426,3,4.0,1,1.61,0.0415,4.69,3,100.00,0,17747,4,14,2,0,1,1,NaN,413.83,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,0.021505,0.021505,1.0,0.700000,0.0,1,0.0,1,-0.081331,0.001267,0.844502,4,5,2
4,4785936,321113,2013-06-08 08:33:16,5,219,NaN,NaN,219,4426,3,4.0,1,1.61,0.5441,0.00,1,90.00,0,5352,4,25,2,0,1,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0,1.0,0.021505,0.021505,1.0,0.700000,0.0,1,0.0,1,89.000000,0.001267,0.356435,6,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246618,4948667,332126,2012-11-01 07:25:28,5,219,NaN,NaN,219,47322,0,3.5,1,2.64,NaN,4.39,31,50.00,0,10080,2,0,2,0,1,1,NaN,313.11,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,0.083333,0.083333,1.0,0.350000,0.0,1,0.0,1,-0.379964,0.001267,0.309352,11,3,2
246619,4948672,332126,2012-11-01 07:25:28,5,219,NaN,NaN,219,100864,0,3.5,1,2.64,NaN,4.25,30,43.00,0,10080,2,0,2,0,1,1,NaN,313.10,0,NaN,NaN,NaN,1.0,0.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,0.083333,0.083333,1.0,0.350000,1.0,1,0.0,1,-0.386638,0.001267,0.309352,11,3,2
246620,4948675,332126,2012-11-01 07:25:28,5,219,NaN,NaN,219,110271,0,1.5,1,1.61,NaN,4.39,32,71.00,0,10080,2,0,2,0,1,1,NaN,311.58,0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,0.000000,0.000000,1.0,0.150000,0.5,1,0.0,1,-0.119548,0.001267,0.309352,11,3,2
246621,4956913,332668,2013-02-20 13:38:12,5,219,NaN,NaN,27,32070,0,0.0,0,3.50,0.0113,6.21,24,74.00,0,18945,6

In [73]:
# df_test.drop(columns=rate_cols+inv_cols+rate_perc_cols+["date_time"], inplace=True)

In [74]:
len(df_test) - 4959183

-4712560

In [75]:
# both datasets have all the added columns
np.setdiff1d(df_train.columns, df_test.columns)

array([], dtype=object)

In [76]:
# df_test.to_csv("data/FE_test_set.csv")
df_test.to_csv("data/train_test_FE.csv")